## **Imports**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
import time

%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

## EDA : Application data

In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

In [ ]:
# training 데이터 피쳐 살펴보기
print('Training data shape: ', app_train.shape)
app_train.head()

# Training data 는 307511개의 데이터가 있고 TARGET을 포함한 122개의 피쳐를 갖고 있다.

In [ ]:
# testing 데이터 피쳐 살펴보기
print('Testing data shape: ', app_test.shape)
app_test.head()

# Testing data는 48744 개의 데이터와 TARGET 이 빠진 121개의 피쳐를 갖고 있다.

In [ ]:
# 'TARGET' 컬럼 살펴보기
app_train['TARGET'].value_counts()

In [ ]:
# TARGET 컬럼 데이터 개수 히스토그램 시각화
app_train['TARGET'].plot.hist();

'''
0은 대출을 제때 상환할 수 있다는 뜻이고 1은 대출상환에 어려움을 갖고 있다는 것을 의미한다. 
'''

In [ ]:
# target 개수 시각화 2
f,ax=plt.subplots(1,2,figsize=(12,6))
app_train.TARGET.value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Distribution of Target')
ax[0].set_ylabel('')
sns.countplot('TARGET',data=app_train,ax=ax[1])
ax[1].set_title('Target count')
plt.show()

# 0이 1보다 훨씬 더 많았고 1은 데이터의 8%만을 차지한다. 비율이 반반이 아니므로 모델링 할 때 가중치 조절이 필요하다. 

In [ ]:
# training 데이터 칼럼 확인
app_train.columns.values

In [ ]:
# column type 수 확인
app_train.dtypes.value_counts()

# int64 와 float64 는 수치형 변수이고 object는 string을 포함한 범주형 변수이다. 나중에 encoding 할 필요가 있어보인다. 

In [ ]:
# target 변수와 다른 변수들간의 상관관계를 살펴보았다. 그리고 sorting 하였다. 
correlations = app_train.corr()['TARGET'].sort_values()

# 양의 상관관계 높은 것 15개, 음의 상관관계 높은 것 15개 나타내기
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

'''
'DAYS_BIRTH','DAYS_EMPLOYED','REGION_RATING_CLIENT_W_CITY'가 양의 상관관계를 보이는 것들 중에 가장 높았다.
'EXT_SOURCE_3','EXT_SOURCE_2','EXT_SOURCE_1'가 음의 상관관계를 보이는 것들 중에 가장 높았다. 
이 변수들을 중점으로 EDA를 할 것이다. 
'''

### EDA1 : 'DAYS_BIRTH'

'DAYS BIRTH' 는 고객의 고객 나이를 신청 일자 기준으로 변환한 값이다. 따라서 음수값이고, 일(days) 단위이다. 

In [ ]:
# DAYS_BIRTH 데이터 살펴보기
app_train['DAYS_BIRTH'].head()

In [ ]:
# 'DAYS_BIRTH' 가 음수이고 년 단위로 보기 위해서 -365로 나눈다. 
(app_train['DAYS_BIRTH'] / -365).describe()

In [ ]:
plt.style.use('fivethirtyeight')

# 고객 나이에 대한 히스토그램 분포를 확인한다. 
plt.hist(app_train['DAYS_BIRTH']/-365, edgecolor='k',bins=25)
plt.title('Age of Client');
plt.xlabel('Age (years)');
plt.ylabel('Count');

# 이상값 없이 나이가 고르게 분포되어 있다. 

In [ ]:
plt.figure(figsize=(10,8))

# 제때 대출을 상환하는 고객의 나이 plot (TARGET=0)
sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/-365,label='target==0')

# 제때 대출을 상환하지못하는 고객의 나이 plot (TARGET=1)
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/-365,label='target==1')

plt.xlabel('Age(years)');
plt.ylabel('Density');
plt.title('Distribution of Ages');

# target==1(빨간색) 의 분포를 보면 20-30대에 기울어 있는것을 볼 수 있다. 이는 젊은 층일수록 대출 상환을 못할 확률이 높다고 유추할 수 있다.
# target==0일때와 1일때의 TARGET과의 분포가 상이한것으로 보아 이 변수는 머신러닝 모델에 유용하게 활용될 것으로 보인다.


In [ ]:
# 나이를 나이대 별로 그룹을 나눠서 target=1(대출 상환이 어려운) 의 평균값을 살펴본다.
# 최소 20 최대 70으로해서 총 10개로 그룹핑하였다. 
# 결과는 '~'이상 '~'미만으로 그룹핑된다.
np.linspace(20,70,num=11)

In [ ]:
# cut() 함수를 사용해서 5살 간격으로 나이대 그룹을 나눠보았고, 각 나이대 별로 대출상환을 못하는 비율을 체크하였다.

age_data=app_train[['TARGET','DAYS_BIRTH']]
age_data['DAYS_BIRTH']=-age_data['DAYS_BIRTH']
age_data['YEARS_BIRTH']=age_data['DAYS_BIRTH']/365
age_data['YEARS_BINNED']=pd.cut(age_data['YEARS_BIRTH'],bins=np.linspace(20,70,num=11))
age_data.head(10)

In [ ]:
# bin으로 groupby 하고 평균 계산하기
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize=(8,8))

# 나이와 target 평균을 bar plot으로 시각화 하였다. 
plt.bar(age_groups.index.astype(str), 100*age_groups['TARGET'])
plt.xticks(rotation=75);
plt.xlabel('Age Group (years)');
plt.ylabel('Failur to Reapy(%)')
plt.title('Failure to Repay by Age Group');

# 젊은층일수록 대출을 상환하지 못하는 것으로 나타났다. 
# 20-25세, 25-30세 30-35세는 각각 약10% 이상 대출을 상환하지 못했고, 55-60세, 60-65세, 65-70세는 5%이하로 대출을 상환하지 못했다.

### EDA2 : 'DAYS_EMPLOYED'

'DAYS_EMPLOYED'는 고객 소득과 관련된 데이터로, 대출 신청전 현 직업 유지기간을 의미한다. 마찬가지로 음수 값을 띄며 일(days) 단위 이다.

In [ ]:
# 데이터 살펴보기 
app_train['DAYS_EMPLOYED'].head()

In [ ]:
# 통계량 확인하기
app_train['DAYS_EMPLOYED'].describe()
# 최댓값이 365243 이고, std 값이 매우 큰 것으로 보아 이상점이 있을 것이라 판단된다. 

In [ ]:
# 365243 이라는 최대값을 갖고 있다. 이상값이라고 판단하기에는 시각화로 보았을 때 무시할 수 없을 정도로 그 수가 상당히 많다.
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

In [ ]:
# 365243 을 이상값으로 보고, 365243인 값과 아닌 값으로 나누어 target 비율을 살펴보았다.

anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]
print('이상값이 아닌 data의 target 평균: %0.2f%%' % (100 * non_anom['TARGET'].mean()))
print('이상값인 data의 target 평균: %0.2f%%' % (100 * anom['TARGET'].mean()))

'''
365243을 갖는 데이터와 아닌 데이터로 나누어서 타겟값의 평균의 100을 곱해서 부채 비율을 보았다. 
이상값으로 보이는 고객들이 대출을 상환하지못할 확률이 5.4%로 더 낮다. 
'''

In [ ]:
'''
이상값은 결측치로 채울 것이다.
이 경우 모든 이상값들이 같은값(365243)을 갖고 있으므로, 다 같은 값으로 채워야 한다. 그리고 나중에 머신러닝 모델에 이 이상값들을 
임의로 채운것에 대해 알려줄 것이다. 이상값을 숫자로 채우지 않고, 새로운 boolean 컬럼을 만들어서 이상값인지 아닌지를 구분할것이다.
'''
# 이상값(365243)인 값에 대해서 True , False로 구분
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# 이상값을 nan값으로 대치
app_train['DAYS_EMPLOYED'].replace({365243:np.nan},inplace=True)

# 히스토그램으로 확인하기
app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram');
plt.xlabel('Days Employment');

# 365243의 값이 nan값으로 대치되었다.

In [ ]:
# test 데이터에도 train 데이터와 동일하게 작업한다.
app_test['DAYS_EMPLOYED_ANOM']=app_test['DAYS_EMPLOYED']==365243
app_test['DAYS_EMPLOYED'].replace({365243:np.nan}, inplace=True)

# True, False로 되어있는 데이터 sum하면 True인것 개수를 카운팅한다.
print('%d 개의 data 중에 testing data에서 %d 개의 이상값이 있다.'%(len(app_test),app_test['DAYS_EMPLOYED_ANOM'].sum()))

### EDA3 : 'REGION_RATING_CLIENT_W_CITY'

'REGION_RATING_CLIENT_W_CITY'는 고객의 거주지를 나타내는 변수로, (1,2,3) 으로 거주 지역을 평가하여 나눈 변수이다. 

In [ ]:
# 데이터 살펴보기
app_train['REGION_RATING_CLIENT_W_CITY'].head(10)

In [ ]:
# catplot 으로 target 변수에 따른 데이터 분포 살펴보기
sns.catplot(x='REGION_RATING_CLIENT_W_CITY', col ='TARGET', data=app_train, kind='count')

# REGION_RATING_CLIENT_W_CITY가 2,3 의 경우 대출상환을 못하는 것으로 판단된다. 밑에서 더 자세히 살펴보겠다.

In [ ]:
# REGION_RATING_CLIENT_W_CITY 에 대해 TARGET 1의 비율 보기
for a in range(3):
    print(len(app_train[(app_train['TARGET']==1)&(app_train['REGION_RATING_CLIENT_W_CITY']==a+1)])
      /len(app_train[app_train['REGION_RATING_CLIENT_W_CITY']==a+1])*100)

'''
REGION_RATING_CLIENT_W_CITY가 1인 고객의 대출상환을 못하는 경우는 4.8%, 2인 고객의 경우는 7.9%, 3인 고객의 경우는 11.4% 로 
REGION_RATING_CLIENT_W_CITY 평가가 3이 나온 경우에 대출상환이 어려울 것이라고 판단될 확률이 가장 높다.
이는 살고 있는 도시의 평가가 안좋을 수록 대출상환을 못할 확률도 높다는 뜻이다.
'''

### EDA 4 : 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'

EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3 는 외부 자원으로 부터 정규화된 스코어로 고객 행동을 나타낸다. 
음의 상관계수 중에 상관도가 가장 높았기에 이 세가지를 중점으로 EDA를 진행하였다.
TARGET 변수와 EXT_SOURCE와의 상관관계와 EXT_SOURCE 서로간의 상관관계를 살펴보았다.

In [ ]:
# 'DAYS_BIRTH' 를 추가하여 상관 관계를 보았다. 'DAYS_BIRTH' 는 app_train 과 app_test 에서 동일하게 년 단위로 바꿔준다.
 
app_train['DAYS_BIRTH'] = app_train['DAYS_BIRTH'] / -365
app_test['DAYS_BIRTH'] = app_test['DAYS_BIRTH'] / -365

ext=app_train[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]
extcorr = ext.corr()
extcorr

In [ ]:
# 상관관계로 히트맵을 그려보았다.
plt.figure(figsize=(8,6))
sns.heatmap(extcorr, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

'''
'EXT_SOURCE'와 'TARGET'는 음의 상관관계에 있으므로, EXT_SOURCE값이 증가할수록 target값이 0이 많은 것으로, 대출 상환을 할 확률이 높다.
'DAYS_BIRTH'는 'EXT_SOURCE_1'와 양의 상관관계에 있는 것으로 보아 이 score중 하나는 고객의 나이일것으로 추정된다.
'''

In [ ]:
# EXT_SOURCE1,2,3을 TARGET값 별로 나눠서 kdeplot 으로 시각화 하였다.
plt.figure(figsize=(10,12))

for i, source in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']):
    # subplot 으로 나누어 주기
    plt.subplot(3,1,i+1)
    
    # target 별로 kdeplot 나타내기
    sns.kdeplot(app_train.loc[app_train['TARGET']==0,source],label='target==0')
    sns.kdeplot(app_train.loc[app_train['TARGET']==1,source],label='target==1')
    
    # title과 x축, y축 지정
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' %source);
    plt.ylabel('Density');

# 높이 조정
plt.tight_layout(h_pad=2.5)

'''
EXT_SOURCE_2 와 EXT_SOURCE_3은 target값에 따라 차이가 나타나는 것을 확인할 수 있다.
target 과의 상관계수가 많이 높지는 않았지만, 
target이 0인지 1인지에 따라 값이 다른것으로 보아 모델에 영향을 주는 주요 변수라고 판단할 수 있다.
'''

### EDA 5 : 범주형 변수

column type 수를 확인 했을 때, string을 포함한 범주형 변수가 있었다. 각 변수마다 target 분포를 확인해 보고 encoding할 필요가 있다.

In [ ]:
# column type 수 확인
app_train.dtypes.value_counts()

# int64 와 float64 는 수치형 변수이고 object는 string을 포함한 범주형 변수이다. 나중에 encoding 할 필요가 있어보인다. 

In [ ]:
# object 변수 list로 바꾸어 확인
object_columns = app_train.dtypes[app_train.dtypes == 'object'].index.tolist()
object_columns

- 'NAME_CONTRACT_TYPE' : 대출유형
- 'CODE_GENDER': 성별
- 'FLAG_OWN_CAR': 차 소유 여부
- 'FLAG_OWN_REALTY': 부동산 소유 여부
- 'NAME_TYPE_SUITE': 동행고객
- 'NAME_INCOME_TYPE': 소득유형
- 'NAME_EDUCATION_TYPE': 고객 교육 레벨
- 'NAME_FAMILY_STATUS': 고객 가족 유형
- 'NAME_HOUSING_TYPE': 주택 유형
- 'OCCUPATION_TYPE': 직업 유형
- 'WEEKDAY_APPR_PROCESS_START': 대출신청 시작요일
- 'ORGANIZATION_TYPE': 고객 직장 유형
- 'FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE' : 고객 거주지역 특정값

이 중에서 대출 상환 여부와 관련될 수 있는 'CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_INCOME_TYPE','NAME_HOUSING_TYPE','OCCUPATION_TYPE'에 대하여 살펴보겠다.

In [ ]:
# 각각의 변수마다 target 분포를 확인하였다.

obj =['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_INCOME_TYPE','NAME_HOUSING_TYPE',
      'OCCUPATION_TYPE']

def show_category_by_target(df, columns):
    for column in columns:
        chart = sns.catplot(x=column, col='TARGET', data=df, kind='count')
        chart.set_xticklabels(rotation=65)

show_category_by_target(app_train, obj)

In [ ]:
# target 값에 따른 분포를 살펴보았다.
cond_1 = (app_train['TARGET'] == 1)
cond_0 = (app_train['TARGET'] == 0)

for a in obj:
    print(a)
    print('\n연체인 경우\n',app_train[cond_1][a].value_counts()/app_train[cond_1].shape[0])
    print('\n연체가 아닌 경우\n',app_train[cond_0][a].value_counts()/app_train[cond_0].shape[0])
    print('----------------------------')

**결과**
- 'CODE_GENDER': 연체가 아닌 경우에 비해 남성이 연체인 경우에 비율이 높게 나타났다. 
- 'FLAG_OWN_CAR': 연체인 경우와 연체가 아닌 경우의 차이가 크지 않다.
- 'FLAG_OWN_REALTY': 연체인 경우와 연체가 아닌 경우의 차이가 크지 않다. 
- 'NAME_INCOME_TYPE': State servant, working, unemployed 비율이 연체일 때 더 크게 나타났다. 
- 'NAME_HOUSING_TYPE': 빌린 집에서 살거나 부모님과 함께 살 때 연체 비율이 더 크게 나타났다.
- 'OCCUPATION_TYPE': 노동자, 드라이버, 보안 직무 종사자들의 연체 비율이 더 크게 나타났다. 

### EDA 6 : 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'AMT_ANNUITY','AMT_GOODS_PRICE'

- 'AMT_CREDIT': 대출금액
- 'AMT_INCOME_TOTAL' : 고객 소득
- 'AMT_ANNUITY' : 월 대출 지급액
- 'AMT_GOODS_PRICE' : 소비자 대출 상품액

In [ ]:
# 상관 관계를 도출해 보았다.
amt=app_train[['TARGET','AMT_CREDIT', 'AMT_INCOME_TOTAL', 'AMT_ANNUITY','AMT_GOODS_PRICE']]
amtcorr = amt.corr()
amtcorr

In [ ]:
# 상관관계로 히트맵을 그려보았다.
plt.figure(figsize=(8,6))
sns.heatmap(amtcorr, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

'''
AMT_CREDIT과 AMT_ANNUITY 의 상관관계는 0.77로 높았다. 월 대출금액이 높으면 대출 금액도 높아지는 것은 당연한 결과이다.
TARGET 과 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'AMT_ANNUITY' 변수들은 모두 음의 상관관계를 나타내었다. 하지만 그 정도는 크지 않다.
'AMT_INCOME_TOTAL'(고객소득)은 'AMT_CREDIT'와 'AMT_ANNUITY'와 양의 상관관계를 갖는다. 
고객의 소득이 클 수록 대출 금액도 커진다는 것을 유추할 수 있다.
'AMT_GOODS_PRICE'는 'AMT_CREDIT'과 'AMT_ANNUITY' 변수와 높은 양의 상관관계를 갖는다. 
'''

In [ ]:
# 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'AMT_ANNUITY'을 TARGET값 별로 나눠서 kdeplot 으로 시각화 하였다.
plt.figure(figsize=(10,12))

for i, source in enumerate(['AMT_CREDIT', 'AMT_INCOME_TOTAL', 'AMT_ANNUITY']):
    # subplot 으로 나누어 주기
    plt.subplot(3,1,i+1)
    
    # target 별로 kdeplot 나타내기
    sns.kdeplot(app_train.loc[app_train['TARGET']==0,source],label='target==0')
    sns.kdeplot(app_train.loc[app_train['TARGET']==1,source],label='target==1')
    
    # title과 x축, y축 지정
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' %source);
    plt.ylabel('Density');

# 높이 조정
plt.tight_layout(h_pad=2.5)

'''
target값에 따라서 큰 차이를 보이지는 않는다. 이는 feature engineering을 통해 target 값 간의 차이를 넓힐 수 있음을 의미할 수 있다.
'''

### 결측값 확인

In [ ]:
# 함수를 만들어 결측값의 유무를 확인한다. 
def missing_values_table(df):
        # 총 결측값
        miss = df.isnull().sum()
        
        # 결측값 비율
        miss_percent = 100 * miss / len(df)
        
        # 표 만들기
        mis_table = pd.concat([miss, miss_percent], axis=1)
        
        # column 이름 다시 설정
        mis_val_table = mis_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # 결측값 비율 높은 순으로 내림차순 하기
        mis_val_table = mis_val_table[
            mis_val_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # 정보 요약
        print ("선택된 데이터프레임은 " + str(df.shape[1]) + "개의 컬럼이 있다.\n"      
            "그중에서 " + str(mis_val_table.shape[0]) +
              " 개는 결측값이 있는 컬럼이다.")
        
        return mis_val_table

In [ ]:
# 결측값 통계
missing_values = missing_values_table(app_train)
missing_values.head(20)

### 데이터 가공 전 training data, testing data 결합

In [ ]:
app_train.shape, app_test.shape

In [ ]:
apps = pd.concat([app_train,app_test])
print(apps.shape)

In [ ]:
apps['TARGET'].value_counts(dropna=False)

## Feature Engineering : Application data

### 범주형 변수 Label Encoding

In [ ]:
# 범주형 변수를 리스트로 만든 후에 인코딩 진행
object_col = apps.dtypes[apps.dtypes == 'object'].index.tolist()
for column in object_col:
    apps[column] = pd.factorize(apps[column])[0]

In [ ]:
apps.info()

### FE1 : 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'AMT_ANNUITY','AMT_GOODS_PRICE'

다음과 같은 새로운 변수를 추가하였다.
- CREDIT_INCOME_PERCENT: 고객의 수입에 대한 대출 금액 비율 
- ANNUITY_INCOME_PERCENT: 고객이 수입에 대한 월 대출 금액 비율
- CREDIT_TERM: 대출 갚는 기간 (월 단위)
- GOODS_CREDIT_RATIO: 총 대출 금액에 대한 대출 상품 금액 비율
- CREDIT_GOODS_DIFF: 총 대출 금액 - 대출 상품 금액
- GOODS_INCOME_RATIO : 고객 수입에 대한 대출 상품 금액 비율

In [ ]:
# 새로운 변수 추가
apps['CREDIT_INCOME_PERCENT'] = apps['AMT_CREDIT'] / apps['AMT_INCOME_TOTAL']
apps['ANNUITY_INCOME_PERCENT'] = apps['AMT_ANNUITY'] / apps['AMT_INCOME_TOTAL']
apps['CREDIT_TERM'] = apps['AMT_ANNUITY'] / apps['AMT_CREDIT']
apps['GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
apps['CREDIT_GOODS_DIFF'] = apps['AMT_CREDIT'] - apps['AMT_GOODS_PRICE']
apps['GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']

### FE2 : 'EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3'

다음과 같은 새로운 변수를 추가 하였다.
- 'APPS_EXT_SOURCE_MEAN': EXT_SOURCE 들의 평균
- 'APPS_EXT_SOURCE_STD' : EXT_SOURCE 들의 표준편차

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].head()

In [ ]:
# column 방향으로 평균 구하기
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)

In [ ]:
# 상관관계가 높은 것에 대해 너무 가공하면 성능이 오히려 떨어질 수 있으므로 mean과 std 만 구하였다.
apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APPS_EXT_SOURCE_MEAN','APPS_EXT_SOURCE_STD']].head(10)

In [ ]:
# 'APPS_EXT_SOURCE_STD'의 NULL 값 평균으로 채우기
apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APPS_EXT_SOURCE_MEAN','APPS_EXT_SOURCE_STD']].head(10)

### FE3 : 'DAYS_BIRTH', 'DAYS_EMPLOYED'

다음과 같은 새로운 변수를 추가하였다.
- 'EMPLOYED_BIRTH_RATIO' : 고객 나이에 대해 일한 날짜 비율
- 'INCOME_EMPLOYED_RATIO' : 직업 유지 기간에 대한 총 수입 비율
- 'INCOME_BIRTH_RATIO' : 고객 나이에 대한 총 수입 비율
- 'CAR_BIRTH_RATIO' : 고객 나이에 대한 소유 차량 연식
- 'CAR_EMPLOYED_RATIO' : 직업 유지 기간에 대한 소유 차량 연식

In [ ]:
# DAYS_BIRTH, DAYS_EMPLOYED 비율로 소득/자산 관련 Feature 가공

apps['EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
apps['INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
apps['INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
apps['CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
apps['CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']

### Null 값 일괄 변환

In [ ]:
apps.isnull().sum().head(100)

## EDA : CREDIT CARD BALANCE data

In [ ]:
ccb = pd.read_csv('../input/home-credit-default-risk/credit_card_balance.csv')

In [ ]:
ccb.head()

In [ ]:
ccb.shape

In [ ]:
# application_train 데이터와 credit_card_balance 데이터 merge
app_ccb = ccb.merge(app_train, left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='outer')
app_ccb.shape

### Null 값 확인

In [ ]:
missing_values = missing_values_table(ccb)
missing_values.head(20)

### SK_ID_CURR당 평균 SK_ID_PREV 건수 확인

In [ ]:
app_ccb.groupby('SK_ID_CURR').count()

In [ ]:
app_ccb.groupby('SK_ID_CURR')['SK_ID_CURR'].count()

In [ ]:
# boxplot 으로 시각화, 평균

sns.boxplot(app_ccb.groupby('SK_ID_CURR')['SK_ID_CURR'].count())
print(app_ccb.groupby('SK_ID_CURR')['SK_ID_CURR'].count().mean())

# 이상값이 많지 않고 평균이 37 정도이다. 

### EDA1 : 범주형 변수

In [ ]:
categorical_features = ccb.select_dtypes(include = ["object"]).columns
numerical_features = ccb.select_dtypes(exclude = ["object"]).columns
print(categorical_features)
print(numerical_features)

In [ ]:
# 'NAME_CONTRACT_STATUS' 의 target 분포 시각화
def plot_re(df,t1='',t2=''):
    f,ax=plt.subplots(1,2,figsize=(18,8))
    df[[t1,t2]].groupby([t1]).count().plot.bar(ax=ax[0],color='Blue')
    ax[0].set_title('count of customer on '+t1)
    sns.countplot(t1,hue=t2,data=df,ax=ax[1],palette="spring")
    ax[1].set_title(t1+': Target 0 vs Target 1')
    # Rotate x-labels
    plt.xticks(rotation=-90)
    a=plt.show()
    return a

plot_re(app_ccb,'NAME_CONTRACT_STATUS','TARGET')

'''
target이 1인 값의 'NAME_CONTRACT_STATUS'(계약 상태)는 Active인 경우가 많았다.
'''

In [ ]:
# crosstab 으로 보면, 0에서 completed 비율이 1보다 더 높은 것을 알 수 있다.
pd.crosstab(app_ccb.TARGET, app_ccb.NAME_CONTRACT_STATUS, dropna=False, normalize='all')

### EDA2 : 수치형 변수

In [ ]:
# 시각화를 위한 함수 만들어 놓기
def show_hist_by_target(df, columns):
    cond_1 = (df['TARGET'] == 1)
    cond_0 = (df['TARGET'] == 0)
    
    for column in columns:
        fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 4), squeeze=False)
        sns.violinplot(x='TARGET', y=column, data=df, ax=axs[0][0] )
        sns.distplot(df[cond_0][column], ax=axs[0][1], label='0', color='blue')
        sns.distplot(df[cond_1][column], ax=axs[0][1], label='1', color='red')   

In [ ]:
# app_train[['SK_ID_CURR', 'TARGET']] 두가지 컬럼만 갖고 옴
# on='SK_ID_CURR' 컬럼 기준으로 조인

app_ccb_target = ccb.merge(app_train[['SK_ID_CURR', 'TARGET']], on='SK_ID_CURR', how='left')
app_ccb_target.shape

In [ ]:
# 수치형 변수 리스트로 만들기
num_columns = app_ccb_target.dtypes[app_ccb_target.dtypes != 'object'].index.tolist()

In [ ]:
# 'SK_ID_PREV','SK_ID_CURR','TARGET' 컬럼 필요없음

num_columns = [column for column in num_columns if column not in ['SK_ID_PREV', 'SK_ID_CURR', 'TARGET']]
num_columns

In [ ]:
# 시각화를 위해 show_hist_by_target 함수 호출

show_hist_by_target(app_ccb_target, num_columns)

'''
결과 : 분포에 있어서 target 0,1이 서로 차이가 조금 나는 변수는 MONTHS_BALANCE, AMT_BALANCE, AMT_CREDIT_LIMIT_ACTUAL, 
AMT_INST_MIN_REGULARITY,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM 이 해당된다. 
하지만 MONTHS_BALANCE는 신청일 기준 잔액 월을 의미하므로 큰 상관관계는 없을 것으로 예상된다. 
이어서 상관관계 분석을 통해 유의미한 변수를 추려내겠다.
'''

### EDA3 :  상관관계 분석

In [ ]:
# target 변수와 다른 변수들간의 상관관계를 살펴보았다. 그리고 sorting 하였다. 
correlations = app_ccb_target.corr()['TARGET'].sort_values()

# 양의 상관관계 높은 것 15개, 음의 상관관계 높은 것 15개 나타내기
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

'''
결과 : AMT_BALANCE ,AMT_TOTAL_RECEIVABLE ,AMT_RECIVABLE 는 target과 양의 상관관계가 있고, 
CNT_INSTALMENT_MATURE_CUM는 음의 상관관계를 갖고 있다. 이 4개의 변수는 위에서 했던 시각화에서 확인할 수 있었듯이 유의미해 보인다.
AMT_BALANCE, AMT_CREDIT_LIMIT_ACTUAL, AMT_INST_MIN_REGULARITY,AMT_RECIVABLE,
AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM 을 더 자세히 살펴보겠다.
'''

### EDA4 : AMT_BALANCE, AMT_CREDIT_LIMIT_ACTUAL, AMT_INST_MIN_REGULARITY,AMT_RECIVABLE, AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM

target 별 분포와 상관관계로 유의미한 변수라고 추려낸 6개의 변수를 더 자세히 살펴보겠다.
(AMT_BALANCE, AMT_CREDIT_LIMIT_ACTUAL, AMT_INST_MIN_REGULARITY,AMT_RECIVABLE,
AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM )

- AMT_BALANCE : 대출 월별 잔액
- AMT_CREDIT_LIMIT_ACTUAL : 월별 카드 허용 한도
- AMT_INST_MIN_REGULARITY : 최소 할부 납입 금액
- AMT_TOTAL_RECEIVABLE : 총 원금 회수 금액
- AMT_RECIVABLE : 대출 원금 회수 금액
- CNT_INSTALMENT_MATURE_CUM : 대출 납부 횟수


In [ ]:
'''
AMT_BALANCE, AMT_CREDIT_LIMIT_ACTUAL, AMT_INST_MIN_REGULARITY,AMT_RECIVABLE,
AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM  시각화
'''
f, ax = plt.subplots(2,3,figsize=(14,14))

# AMT_BALANCE
sns.distplot(app_ccb.AMT_BALANCE.dropna(), kde=True, color='g',
            ax=ax[0,0]).set_title('AMT_BALANCE')
# AMT_CREDIT_LIMIT_ACTUAL
sns.distplot(app_ccb.AMT_CREDIT_LIMIT_ACTUAL.dropna(), kde=True, color='b',
            ax=ax[0,1]).set_title('AMT_CREDIT_LIMIT_ACTUAL')
# AMT_INST_MIN_REGULARITY
sns.distplot(app_ccb.AMT_INST_MIN_REGULARITY.dropna(), kde=True, color='r',
            ax=ax[0,2]).set_title('AMT_INST_MIN_REGULARITY')
# CNT_INSTALMENT_MATURE_CUM
sns.distplot(app_ccb.CNT_INSTALMENT_MATURE_CUM.dropna(), kde=True, color='g',
            ax=ax[1,0]).set_title('CNT_INSTALMENT_MATURE_CUM')
# AMT_INST_MIN_REGULARITY
sns.distplot(app_ccb.AMT_INST_MIN_REGULARITY.dropna(), kde=True, color='b',
            ax=ax[1,1]).set_title('AMT_INST_MIN_REGULARITY')
# AMT_CREDIT_LIMIT_ACTUAL
sns.distplot(app_ccb.AMT_CREDIT_LIMIT_ACTUAL.dropna(), kde=True, color='r',
            ax=ax[1,2]).set_title('AMT_CREDIT_LIMIT_ACTUAL')



In [ ]:
# mean, median, count 타겟별로 조사해보기

print(app_ccb_target.groupby('TARGET').agg({'AMT_BALANCE': ['mean', 'median', 'count','sum','max']}))
print(app_ccb_target.groupby('TARGET').agg({'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'median', 'count','sum','max']}))
print(app_ccb_target.groupby('TARGET').agg({'AMT_INST_MIN_REGULARITY': ['mean', 'median', 'count','sum','max']}))
print(app_ccb_target.groupby('TARGET').agg({'CNT_INSTALMENT_MATURE_CUM': ['mean', 'median', 'count','sum','max']}))
print(app_ccb_target.groupby('TARGET').agg({'AMT_INST_MIN_REGULARITY': ['mean', 'median', 'count','sum','max']}))
print(app_ccb_target.groupby('TARGET').agg({'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'median', 'count','sum','max']}))

'''
AMT_BALANCE, AMT_CREDIT_LIMIT_ACTUAL, AMT_INST_MIN_REGULARITY,
AMT_CREDIT_LIMIT_ACTUAL,AMT_INST_MIN_REGULARITY 는 TARGET=1일 경우에 mean이 높다.

CNT_INSTALMENT_MATURE_CUM는 타겟이 0인 경우에 평균이 더 높다.
max 값은 0이 모두 다 크다. 
'''

## Feature Engineering : CREDIT CARD BALANCE data

In [ ]:
# 변수로 받아서 ccb_group에 저장
ccb_group = app_ccb_target.groupby('SK_ID_CURR') 
ccb_group.head()

In [ ]:
agg_dict = {
    'SK_ID_CURR':['count'],
    'AMT_BALANCE':['mean', 'max', 'sum'],
    'AMT_CREDIT_LIMIT_ACTUAL':['mean', 'max', 'sum'], 
    'AMT_INST_MIN_REGULARITY':['mean', 'max', 'sum'],
    'CNT_INSTALMENT_MATURE_CUM':['mean', 'max', 'sum'],
    'AMT_CREDIT_LIMIT_ACTUAL':['mean', 'max', 'sum'],
    'AMT_INST_MIN_REGULARITY':['mean', 'max', 'sum']
}

ccb_amt_agg = ccb_group.agg(agg_dict)
ccb_amt_agg.head()
ccb_amt_agg.shape

In [ ]:
# MultiIndex로 되어 있는 컬럼명 확인

ccb_amt_agg.columns

In [ ]:
# 컬럼명 변경
ccb_amt_agg.columns = ['CCB_' + ('_').join(column).upper() for column in ccb_amt_agg.columns.ravel()]
ccb_amt_agg.head()

In [ ]:
ccb_amt_agg=ccb_amt_agg.reset_index()
ccb_amt_agg

In [ ]:
ccb_amt_agg=ccb_amt_agg.drop(['CCB_SK_ID_CURR_COUNT'],axis=1)
ccb_amt_agg

In [ ]:
apps = apps.merge(ccb_amt_agg, left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='left')
app_ccb.shape

### 범주형 변수 인코딩

In [ ]:
# 범주형 변수를 리스트로 만든 후에 인코딩 진행
object_col = apps.dtypes[apps.dtypes == 'object'].index.tolist()
for column in object_col:
    apps[column] = pd.factorize(apps[column])[0]

## Modeling

In [ ]:
apps

In [ ]:
# 데이터 train, test 로 나누기
apps_train = apps[~apps['TARGET'].isnull()]
apps_test = apps[apps['TARGET'].isnull()]
apps.shape, apps_train.shape, apps_test.shape

In [ ]:
from sklearn.model_selection import train_test_split

ftr_app = apps_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = apps_train['TARGET']

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 100)

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))

In [ ]:
# 학습된 Classifier를 이용하여 테스트 데이터 예측하고 결과를 Kaggle로 Submit 수행
preds = clf.predict_proba(apps_test.drop(['SK_ID_CURR', 'TARGET'], axis=1))[:, 1 ]

In [ ]:
apps_test['TARGET'] = preds
apps_test[['SK_ID_CURR', 'TARGET']].to_csv('apps_baseline05.csv', index=False)